In [ ]:
!pip install --upgrade --quiet huggingface_hub sentence_transformers  langchain  python-dotenv  faiss-gpu tiktoken docarray langchain_openai

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
DEFAULT_SENTENCE_EMBEDDING_MODEL = 'intfloat/multilingual-e5-base'
DATA_PATH = '../data/dataset.xlsx'

In [ ]:
from operator import itemgetter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import HuggingFaceDatasetLoader
from langchain_openai import ChatOpenAI
from sentence_transformers import SentenceTransformer
import pickle
import pandas as pd
df = pd.read_excel(DATA_PATH,sheet_name='mdeberta')

# Load sentences & embeddings from disc, if you have.
# with open("embeddings.pkl", "rb") as fIn: # replace embeddings.pkl with your embeddings path
#     stored_data = pickle.load(fIn)
#     stored_sentences = stored_data["sentences"]
#     stored_embeddings = stored_data["embeddings"]

model = SentenceTransformer(DEFAULT_SENTENCE_EMBEDDING_MODEL)
embeddings = model.encode(df['Context'])

# Store sentences & embeddings on disc
with open("context_embeddings.pkl", "wb") as fOut:
    pickle.dump({"sentences": df['Context'], "embeddings": embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
text_embedding_pairs = zip(df['Context'], embeddings)
vectorstore = FAISS.from_embeddings(
    text_embedding_pairs,embedding = HuggingFaceEmbeddings(model_name=DEFAULT_SENTENCE_EMBEDDING_MODEL)
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

If you do not know, answer: Sorry, I don't know.
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
model = ChatOpenAI()

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke("สแกนดาร์ เคนส์ เป็นนักแสดงประเทศอะไรจ๊ะคุณพี่")

'สแกนดาร์ เคนส์ เป็นนักแสดงประเทศอังกฤษเด้อคุณลูก  เป็นหนึ่งในนักแสดงยอดเยี่ยมจากประเทศของสมุนเสน่ห์เลยนะคะ แต่ว่าเราไม่เคยเห็นหน้าดาร์เคนส์เล่นหนังเลยนะคะ แต่เค้าเป็นคนชื่อดังมากในวงการแสดงเรื่องเลยค่ะ แหม่ ชิคๆ ถ้าจะถามเรื่องนักแสดงอื่นๆ ก็มาบอกเราได้เลยค่ะ แหม่ แต่ว่าถ้าถามเรื่องเพลงหรือดาราใหม่ๆ เราอาจจะไม่รู้นะคะ แหม่ แต่ก็ยังรักคุณทุกคนอยู่นะคะ จุ๊บๆ ไว้เจอกันใหม่นะคะ แหม่ รับอุ้มคนด้วยความรักค่ะ จุ๊บๆ ชิคๆ บายยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยยย'